- Author: Patrick Mthisi
- Contact details: patrickmthisi@hotmail.com

# Understanding the Effects of Air Quality on Public Health

The dataset for the project was sourced from: https://www.kaggle.com/datasets/rabieelkharoua/air-quality-and-health-impact-dataset

About Dataset:

This dataset contains comprehensive information on the air quality and its impact on public health for 5,811 records. It includes variables such as air quality index (AQI), concentrations of various pollutants, weather conditions, and health impact metrics. The target variable is the health impact class, which categorizes the health impact based on the air quality and other related factors.

#  Objectives:

In this short study, we aim to investigate the impact of air quality on public health using Google's TabNet classifier on a highly imbalanced multi-class dataset. The following concepts can be gleaned from this study:
 - Performing Bayesian hyperparameter optimisation using Optuna to get an optimal set of hyperparameters for the weighted TabNet model.
 - Constructing a TabNet ensemble classifier.

# Import Libraries:

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, balanced_accuracy_score, confusion_matrix, average_precision_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
import sweetviz
import dtale
import optuna
import sys
import contextlib
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", message="Best weights from best epoch are automatically used!")

sns.set_theme(context='notebook', 
              style='whitegrid',
              palette='colorblind', 
              color_codes=True, 
              rc={'figure.figsize': [10, 6]})

# Load dataset and perform EDA:

In [2]:
df = pd.read_csv("air_quality_health_impact_data.csv", na_values=np.nan)
dtale.show(df, host='localhost')

# Training, validation, and test data split:

In [3]:
TARGET = "HealthImpactClass"
RANDOM_STATE = 42
TEST_SIZE = 0.15
VALID_SIZE = 0.15

X = df.drop(columns=["RecordID"]).copy()
y = X.pop(TARGET).astype(int)

num_features = X.select_dtypes("number").columns.to_list()

X_train, X_temp, y_train, y_temp  = train_test_split(
    X, y, 
    test_size=(TEST_SIZE + VALID_SIZE), 
    random_state=RANDOM_STATE, 
    stratify=y
)

new_test_size = TEST_SIZE / (TEST_SIZE + VALID_SIZE)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp,
    test_size=new_test_size, 
    stratify=y_temp,
    random_state=RANDOM_STATE
)

print("Percentage of data in each set ----")
print(f"Train: {100 * len(X_train) / len(X):.2f}%")
print(f"Valid: {100 * len(X_valid) / len(X):.2f}%")
print(f"Test : {100 * len(X_test) / len(X):.2f}%")
print("")
print("Class distribution in each set ----")
print(f"Train: {y_train.value_counts(normalize=True).values}")
print(f"Valid: {y_valid.value_counts(normalize=True).values}")
print(f"Test : {y_test.value_counts(normalize=True).values}")

classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
sample_weights = np.array([class_weights[cls] for cls in y_train])
print("\nSample weights ----")
print(class_weights)

Percentage of data in each set ----
Train: 69.99%
Valid: 15.01%
Test : 15.01%

Class distribution in each set ----
Train: [0.8273912  0.099582   0.04696336 0.01647406 0.00958938]
Valid: [0.82683486 0.09977064 0.04701835 0.01605505 0.0103211 ]
Test : [0.82798165 0.09977064 0.04701835 0.01605505 0.00917431]

Sample weights ----
[ 0.24172363  2.00839506  4.25863874 12.14029851 20.85641026]


# Hyperparameter optimisation of TabNet classifier:

In [5]:
# implement Bayesian hyperparameter optimisation
def objective(trial):
    # sample hyperparameters:
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    gamma = trial.suggest_float("gamma", 0.1, 2.0)
    step_size = trial.suggest_int("step_size", 10, 50)
    n_d = trial.suggest_int("n_d", 5, 13)
    n_a = trial.suggest_int("n_a", 5, 13)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-2, log=True)
    momentum = trial.suggest_float("momentum", 0.1, 1.0)
    
    # create TabNet model:
    clf = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=lr),
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        scheduler_params={"step_size": step_size, "gamma": gamma},
        n_a=n_a,
        n_d=n_d,
        lambda_sparse=lambda_sparse,
        momentum=momentum,
        mask_type="sparsemax",
        seed=RANDOM_STATE,
        verbose=0,
    )
    
    # fit model
    clf.fit(
        X_train=X_train.values,
        y_train=y_train.values,
        eval_set=[
            (X_train.values, y_train.values),
            (X_valid.values, y_valid.values)
        ],
        eval_name=["train", "valid"],
        eval_metric=["balanced_accuracy"],
        max_epochs=200,
        patience=20,
        batch_size=128,
        virtual_batch_size=16,
        weights=sample_weights, 
    )

    # predict and evaluate:
    yhat = clf.predict(X_valid.values)
    score = balanced_accuracy_score(y_valid.values, yhat)
    
    return score

# run optimization:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, n_jobs=-1)

print("Best hyperparameters:", study.best_params)

[I 2025-07-21 18:06:39,514] A new study created in memory with name: no-name-364c6f12-b15d-45a7-88eb-9af25a24937e



Early stopping occurred at epoch 25 with best_epoch = 5 and best_valid_balanced_accuracy = 0.25381


[I 2025-07-21 18:25:22,743] Trial 7 finished with value: 0.2538121769375904 and parameters: {'lr': 9.769238918980915e-05, 'gamma': 0.1207145446250604, 'step_size': 19, 'n_d': 12, 'n_a': 11, 'lambda_sparse': 0.00010910232817561727, 'momentum': 0.1884043864011105}. Best is trial 7 with value: 0.2538121769375904.



Early stopping occurred at epoch 30 with best_epoch = 10 and best_valid_balanced_accuracy = 0.62985


[I 2025-07-21 18:29:09,577] Trial 0 finished with value: 0.6298513587268925 and parameters: {'lr': 0.003175196233119422, 'gamma': 1.0394598524078422, 'step_size': 49, 'n_d': 9, 'n_a': 8, 'lambda_sparse': 0.0001201212179796293, 'momentum': 0.7038571051510726}. Best is trial 0 with value: 0.6298513587268925.


...


Early stopping occurred at epoch 75 with best_epoch = 55 and best_valid_balanced_accuracy = 0.58123


[I 2025-07-22 02:52:37,521] Trial 98 finished with value: 0.5812339728447741 and parameters: {'lr': 0.0023291818243258494, 'gamma': 0.2939147108594704, 'step_size': 19, 'n_d': 11, 'n_a': 6, 'lambda_sparse': 0.0003883261799334423, 'momentum': 0.8283956755925035}. Best is trial 91 with value: 0.6648501747344702.



Early stopping occurred at epoch 76 with best_epoch = 56 and best_valid_balanced_accuracy = 0.62266


[I 2025-07-22 02:52:54,062] Trial 99 finished with value: 0.6226614075887758 and parameters: {'lr': 0.0023425227331311067, 'gamma': 1.144383508724929, 'step_size': 13, 'n_d': 11, 'n_a': 6, 'lambda_sparse': 0.00021018842181296047, 'momentum': 0.37337334300071984}. Best is trial 91 with value: 0.6648501747344702.


Best hyperparameters: {'lr': 0.024661276949600026, 'gamma': 1.899995840281072, 'step_size': 27, 'n_d': 6, 'n_a': 6, 'lambda_sparse': 0.0002195874329439208, 'momentum': 0.15003590502748304}


# Create an ensemble classifier:

In [16]:
final_params = {
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=study.best_params["lr"]),
    "scheduler_fn": torch.optim.lr_scheduler.StepLR,
    "scheduler_params": {
        "step_size": study.best_params["step_size"], 
        "gamma": study.best_params["gamma"]
    },
    "n_d": study.best_params["n_d"],
    "n_a": study.best_params["n_a"],
    "momentum": study.best_params["momentum"],
    "lambda_sparse": study.best_params["lambda_sparse"],
    "seed": RANDOM_STATE,
    "mask_type": "sparsemax",
    "verbose": 0, 
}

models = []
pred_probs = []
N_ENSEMBLES = 10

for i in range(N_ENSEMBLES):
    print(f"\nTraining model {i+1}/{N_ENSEMBLES} with seed={RANDOM_STATE+i}...")
    final_params["seed"] = RANDOM_STATE + i
    model_params = final_params
    
    model = TabNetClassifier(**model_params)
    model.fit(
        X_train=X_train.values,
        y_train=y_train.values,
        eval_set=[
            (X_valid.values, y_valid.values)
        ],
        eval_name=["valid"],
        eval_metric=["balanced_accuracy"],
        max_epochs=200,
        patience=30,
        batch_size=128,
        virtual_batch_size=16,
        weights=sample_weights,
    )
    
    models.append(model)
    preds = model.predict_proba(X_valid.values)
    pred_probs.append(preds)
    
avg_preds = np.mean(pred_probs, axis=0)
final_preds = np.argmax(avg_preds, axis=1)

print("\n=== Ensemble Evaluation ===")
print("Balanced accuracy", balanced_accuracy_score(y_valid.values, final_preds))
print(classification_report(y_valid.values, final_preds, digits=4))


Training model 1/10 with seed=42...

Early stopping occurred at epoch 83 with best_epoch = 53 and best_valid_balanced_accuracy = 0.61985

Training model 2/10 with seed=43...

Early stopping occurred at epoch 50 with best_epoch = 20 and best_valid_balanced_accuracy = 0.6126

Training model 3/10 with seed=44...

Early stopping occurred at epoch 120 with best_epoch = 90 and best_valid_balanced_accuracy = 0.624

Training model 4/10 with seed=45...

Early stopping occurred at epoch 93 with best_epoch = 63 and best_valid_balanced_accuracy = 0.63512

Training model 5/10 with seed=46...

Early stopping occurred at epoch 80 with best_epoch = 50 and best_valid_balanced_accuracy = 0.65303

Training model 6/10 with seed=47...

Early stopping occurred at epoch 95 with best_epoch = 65 and best_valid_balanced_accuracy = 0.64048

Training model 7/10 with seed=48...

Early stopping occurred at epoch 57 with best_epoch = 27 and best_valid_balanced_accuracy = 0.62346

Training model 8/10 with seed=49...

# Model evaluation using out-of-sample test dataset:

In [73]:
test_classes = np.unique(y_test)
test_class_weights = compute_class_weight(class_weight="balanced", classes=test_classes, y=y_test)
test_sample_weights = np.array([test_class_weights[list(test_classes).index(cls)] for cls in y_test])

print("\nSample weights ----")
print(test_class_weights)

preds_probs = []
for model in models:
    preds = model.predict_proba(X_test.values)
    preds_probs.append(preds)

avg_preds = np.mean(preds_probs, axis=0)
final_preds = np.argmax(avg_preds, axis=1)  

print("\n=== Ensemble Evaluation ===")
y_test_bin = label_binarize(y_test.values, classes=test_classes)
print("Average precision: %.4f" % average_precision_score(
    y_test_bin, avg_preds,
    average="weighted",
    sample_weight=test_sample_weights
))

print("ROC AUC: %.4f" % roc_auc_score(
    y_test_bin, avg_preds,
    average="weighted",
    multi_class="ovr",
    sample_weight=test_sample_weights
))

print(classification_report(y_test.values, final_preds, digits=4))



Sample weights ----
[ 0.24155125  2.0045977   4.25365854 12.45714286 21.8       ]

=== Ensemble Evaluation ===
Average precision: 0.6302
ROC AUC: 0.8266
              precision    recall  f1-score   support

           0     0.9682    0.8019    0.8773       722
           1     0.8478    0.8966    0.8715        87
           2     0.8974    0.8537    0.8750        41
           3     1.0000    0.4286    0.6000        14
           4     0.0146    0.2500    0.0276         8

    accuracy                         0.8028       872
   macro avg     0.7456    0.6461    0.6503       872
weighted avg     0.9446    0.8028    0.8643       872



2025-07-22 13:16:28,998 - INFO     - Executing shutdown due to inactivity...
2025-07-22 13:16:29,112 - INFO     - Executing shutdown...
2025-07-22 13:16:29,115 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer


# Conclusion and recommendation...

The dataset exhibited severe class imbalance, with some classes representing less than 1% of the total samples, as highlighted in the EDA file. To address this, a weighted TabNet classifier was employed to ensure the model gave appropriate attention to minority classes. Since accuracy is an unreliable metric in such contexts, we instead optimized the model using balanced accuracy, which better reflects performance across all classes. An ensemble of 10 TabNet models was then constructed to enhance generalization and robustness. On the out-of-sample test set, the ensemble achieved a weighted average precision of 0.6302 and a ROC AUC of 0.8266, demonstrating its ability to correctly classify instances from under-represented classes.

  